## Padding video streams with `tf.data`.

In [1]:
import os
import cv2
import imageio
import numpy as np
from tqdm.notebook import tqdm

import tensorflow as tf

2022-10-30 11:44:02.860147: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-30 11:44:03.374268: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-30 11:44:03.374322: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-30 11:44:03.471026: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-30 11:44:05.845563: W tensorflow/stream_executor/platform/de

In [2]:
video_path = '../inputs/HorseRace/'

## Create frames from each video file.

In [3]:
def format_frames(frame, output_size):
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return frame

In [36]:
def frames_from_video_file(video_path, n_frames = 10, output_size = (224,224), frame_step = 15):
    # Read each video frame by frame
    result = []
    src = cv2.VideoCapture(str(video_path))  

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

    need_length = 1 + (n_frames - 1) * frame_step

    if need_length > video_length:
        start = 0
    else:
        max_start = video_length - need_length
        start = np.random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)
    ret, frame = src.read()
    result.append(format_frames(frame, output_size))

    for _ in range(n_frames - 1):
        if ret:
            frame = tf.image.convert_image_dtype(frame, tf.float32)
            frame = tf.image.resize_with_pad(frame, *output_size)
            result.append(frame)
        else:
            result.append(np.zeros_like(result[0]))
    src.release()
    result = np.array(result)[..., [2, 1, 0]]

    return result

In [37]:
video_path = '../inputs/videos/vtest.avi'

sample_video = frames_from_video_file(video_path)
sample_video.shape

(10, 224, 224, 3)

In [47]:
data_tf = tf.data.Dataset.list_files('../inputs/HorseRace/*.avi')

for element in data_tf.take(3):
    print(element.numpy().decode('utf-8'))

../inputs/HorseRace/v_HorseRace_g06_c05.avi
../inputs/HorseRace/v_HorseRace_g04_c03.avi


In [39]:
sample_avi = '../inputs/HorseRace/v_HorseRace_g06_c05.avi'

sample_avi = frames_from_video_file(sample_avi)
sample_avi.shape

(10, 224, 224, 3)

In [41]:
data_tf = tf.data.Dataset.from_tensors(sample_avi)
data_tf.element_spec

TensorSpec(shape=(10, 224, 224, 3), dtype=tf.float32, name=None)

In [38]:
data_tf = data_tf.map(frames_from_video_file, num_parallel_calls = tf.data.AUTOTUNE)
data_tf.element_spec

ValueError: in user code:

    File "/tmp/ipykernel_9499/3424584958.py", line 18, in frames_from_video_file  *
        result.append(format_frames(frame, output_size))
    File "/tmp/ipykernel_9499/2617991647.py", line 2, in format_frames  *
        frame = tf.image.convert_image_dtype(frame, tf.float32)

    ValueError: None values not supported.


In [49]:
list_1 = [[2, 3, 4], [5, 6]]
list_1_tf = tf.data.Dataset.from_tensor_slices(list_1)
list_1_tf.element_spec

ValueError: Can't convert non-rectangular Python sequence to Tensor.